In [1]:
import aiohttp
import asyncio
import requests
from datetime import datetime
import pandas as pd
import time 
def get_binance_symbols():
    api_url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
        
        symbols_data = response.json()
        
        # Extraction des symboles avec USDT comme quote asset
        symbols = [symbol['symbol'] for symbol in symbols_data['symbols'] if symbol['quoteAsset'] == 'USDT']
        
        return symbols
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")
        return None
def get_klines(symbol):
    api_url = f'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': '1m',  # Klines à la minute
        'limit': 60  # Nombre de bougies à récupérer
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
        
        klines_data = response.json()
        
        # Extraction des données nécessaires (par exemple, le temps de clôture)
        klines = [{'time': kline[0], 'close': kline[4]} for kline in klines_data]
        
        return klines
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la récupération des klines pour le symbole {symbol} : {e}")
        return None
def convert_timestamp_to_date(timestamp):
    # Convertit le timestamp en format de date lisible
    return datetime.utcfromtimestamp(timestamp / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

# Appel de la fonction pour récupérer les symboles
all_symbols = get_binance_symbols()

# Affichage des symboles
#on affiche que les 10 premiers tickers 
if all_symbols:
    print("Liste des symboles disponibles sur Binance :")
    for symbol in all_symbols[:10]:
        print(symbol)


Liste des symboles disponibles sur Binance :
BTCUSDT
ETHUSDT
BNBUSDT
BCCUSDT
NEOUSDT
LTCUSDT
QTUMUSDT
ADAUSDT
XRPUSDT
EOSUSDT


## Méthode séquentielle

In [4]:
import time
# Création d'un DataFrame pour stocker les données

data = {'Symbol': [], 'Time': [], 'Close': []}
#Calcul du temps d'éxécution
debut_seq= time.time()
# Récupération des données pour chaque symbole


if all_symbols[:10]:
    for symbol in all_symbols[:10]:
        klines_data = get_klines(symbol)
        
        # Ajout des données au DataFrame
        if klines_data:
            for kline in klines_data:
                timestamp = convert_timestamp_to_date(kline['time'])
                data['Symbol'].append(symbol)
                data['Time'].append(timestamp)
                data['Close'].append(kline['close'])

# Création du DataFrame
df = pd.DataFrame(data)
fin_seq= time.time()
temps_seq=fin_seq-debut_seq
# Affichage du DataFrame
df
temps_seq

3.050379753112793

## méthode Asynchrone

In [13]:
import aiohttp
import asyncio
import time
debut_async= time.time()
async def get_binance_symbols():
    api_url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    async with aiohttp.ClientSession() as session:
        async with session.get(api_url) as response:
            #response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
            symbols_data = await response.json()
        
        # Extraction des symboles
        symbols = [symbol['symbol'] for symbol in symbols_data['symbols']]
        
        return symbols

async def get_klines(session, symbol):
    api_url = f'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': '1m',  # Klines à la minute
        'limit': 60  # Nombre de bougies à récupérer
    }

    async with session.get(api_url, params=params) as response:
        response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
        klines_data = await response.json()
        
        # Extraction des données nécessaires (par exemple, le temps de clôture)
        klines = [{'time': kline[0], 'close': kline[4]} for kline in klines_data]
        
        return klines


#async def main():
# Appel de la fonction pour récupérer les symboles de manière asynchrone
all_symbols = await get_binance_symbols()

# Récupération des données pour chaque symbole de manière asynchrone
if all_symbols[:10]:
    async with aiohttp.ClientSession() as session:
        tasks = [get_klines(session, symbol) for symbol in all_symbols[:10]]
        klines_data = await asyncio.gather(*tasks)

        # Ajout des données au DataFrame
        for symbol, klines in zip(all_symbols[:10], klines_data):
            for kline in klines:
                timestamp = convert_timestamp_to_date(kline['time'])
                data['Symbol'].append(symbol)
                data['Time'].append(timestamp)
                data['Close'].append(kline['close'])

# Création du DataFrame
df = pd.DataFrame(data)
fin_async= time.time()
temps_async=fin_async-debut_async
# Affichage du DataFrame
df
temps_async

1.6530981063842773

## Méthode Multithreading

In [11]:
import threading
import requests
import time 
from datetime import datetime
debut_Mthread=time.time()
debut_async= time.time()
def get_binance_symbols():
    api_url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
        
        symbols_data = response.json()
        
        # Extraction des symboles
        symbols = [symbol['symbol'] for symbol in symbols_data['symbols']]
        
        return symbols
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")
        return None

def get_klines(symbol):
    api_url = f'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': '1m',  # Klines à la minute
        'limit': 60  # Nombre de bougies à récupérer
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
        
        klines_data = response.json()
        
        # Extraction des données nécessaires (par exemple, le temps de clôture)
        klines = [{'time': kline[0], 'close': kline[4]} for kline in klines_data]
        
        return klines
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la récupération des klines pour le symbole {symbol} : {e}")
        return None


def process_symbol(symbol):
    klines_data = get_klines(symbol)
    
    # Ajout des données au DataFrame
    if klines_data:
        data = {'Symbol': [], 'Time': [], 'Close': []}
        for kline in klines_data:
            timestamp = convert_timestamp_to_date(kline['time'])
            data['Symbol'].append(symbol)
            data['Time'].append(timestamp)
            data['Close'].append(kline['close'])

        # Création du DataFrame
        df = pd.DataFrame(data)

        # Affichage du DataFrame
        print(df)
        print('\n' + '='*30 + '\n')  # Séparateur entre les symboles

# Appel de la fonction pour récupérer les symboles
all_symbols = get_binance_symbols()

# Utilisation du module threading pour récupérer les données à l'aide de threads
if all_symbols[:10]:
    threads = [threading.Thread(target=process_symbol, args=(symbol,)) for symbol in all_symbols[:10]]

    # Démarrage des threads
    for thread in threads:
        thread.start()

    # Attente de la fin de tous les threads
    for thread in threads:
        thread.join()

fin_Mthread= time.time()
temps_Mthread=fin_Mthread-debut_Mthread
temps_Mthread

    Symbol                 Time       Close
0   SNTETH  2023-11-18 14:16:00  0.00002060
1   SNTETH  2023-11-18 14:17:00  0.00002060
2   SNTETH  2023-11-18 14:18:00  0.00002060
3   SNTETH  2023-11-18 14:19:00  0.00002060
4   SNTETH  2023-11-18 14:20:00  0.00002060
5   SNTETH  2023-11-18 14:21:00  0.00002060
6   SNTETH  2023-11-18 14:22:00  0.00002060
7   SNTETH  2023-11-18 14:23:00  0.00002060
8   SNTETH  2023-11-18 14:24:00  0.00002060
9   SNTETH  2023-11-18 14:25:00  0.00002060
10  SNTETH  2023-11-18 14:26:00  0.00002060
11  SNTETH  2023-11-18 14:27:00  0.00002060
12  SNTETH  2023-11-18 14:28:00  0.00002074
13  SNTETH  2023-11-18 14:29:00  0.00002074
14  SNTETH  2023-11-18 14:30:00  0.00002074
15  SNTETH  2023-11-18 14:31:00  0.00002074
16  SNTETH  2023-11-18 14:32:00  0.00002074
17  SNTETH  2023-11-18 14:33:00  0.00002074
18  SNTETH  2023-11-18 14:34:00  0.00002074
19  SNTETH  2023-11-18 14:35:00  0.00002074
20  SNTETH  2023-11-18 14:36:00  0.00002074
21  SNTETH  2023-11-18 14:37:00 

    Symbol                 Time       Close
0   BNTETH  2023-11-18 14:16:00  0.00039340
1   BNTETH  2023-11-18 14:17:00  0.00039510
2   BNTETH  2023-11-18 14:18:00  0.00039510
3   BNTETH  2023-11-18 14:19:00  0.00039690
4   BNTETH  2023-11-18 14:20:00  0.00039600
5   BNTETH  2023-11-18 14:21:00  0.00039600
6   BNTETH  2023-11-18 14:22:00  0.00039600
7   BNTETH  2023-11-18 14:23:00  0.00039600
8   BNTETH  2023-11-18 14:24:00  0.00039510
9   BNTETH  2023-11-18 14:25:00  0.00039510
10  BNTETH  2023-11-18 14:26:00  0.00039510
11  BNTETH  2023-11-18 14:27:00  0.00039510
12  BNTETH  2023-11-18 14:28:00  0.00039330
13  BNTETH  2023-11-18 14:29:00  0.00039330
14  BNTETH  2023-11-18 14:30:00  0.00039330
15  BNTETH  2023-11-18 14:31:00  0.00039330
16  BNTETH  2023-11-18 14:32:00  0.00039330
17  BNTETH  2023-11-18 14:33:00  0.00039330
18  BNTETH  2023-11-18 14:34:00  0.00039230
19  BNTETH  2023-11-18 14:35:00  0.00039230
20  BNTETH  2023-11-18 14:36:00  0.00039230
21  BNTETH  2023-11-18 14:37:00 

1.6783242225646973

## Méthode multiprocessing

In [12]:
import multiprocessing
import requests
from datetime import datetime
debut_Mproc= time.time()
def get_binance_symbols():
    api_url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
        
        symbols_data = response.json()
        
        # Extraction des symboles
        symbols = [symbol['symbol'] for symbol in symbols_data['symbols']]
        
        return symbols
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite : {e}")
        return None

def get_klines(symbol):
    api_url = f'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': '1m',  # Klines à la minute
        'limit': 60  # Nombre de bougies à récupérer
    }

    try:
        response = requests.get(api_url, params=params)
        response.raise_for_status()  # Vérifie si la requête a réussi (code 200)
        
        klines_data = response.json()
        
        # Extraction des données nécessaires (par exemple, le temps de clôture)
        klines = [{'time': kline[0], 'close': kline[4]} for kline in klines_data]
        
        return klines
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la récupération des klines pour le symbole {symbol} : {e}")
        return None

def convert_timestamp_to_date(timestamp):
    # Convertit le timestamp en format de date lisible
    return datetime.utcfromtimestamp(timestamp / 1000.0).strftime('%Y-%m-%d %H:%M:%S')

def process_symbol(symbol):
    klines_data = get_klines(symbol)
    
    # Affichage des données (par exemple, le temps de clôture des bougies)
    if klines_data:
        print(f"Données pour le symbole {symbol}:")
        for kline in klines_data:
            time = convert_timestamp_to_date(kline['time'])
            print(f"Temps: {time}, Clôture: {kline['close']}")
        print('\n' + '='*30 + '\n')  # Séparateur entre les symboles

# Appel de la fonction pour récupérer les symboles
all_symbols = get_binance_symbols()

# Utilisation du module multiprocessing pour récupérer les données à l'aide de processus
if all_symbols:
    processes = [multiprocessing.Process(target=process_symbol, args=(symbol,)) for symbol in all_symbols]

    # Démarrage des processus
    for process in processes:
        process.start()

    # Attente de la fin de tous les processus
    for process in processes:
        process.join()
#Calcul du temps du processus
fin_Mproc= time.time()
temps_Mproc=fin_Mproc-debut_Mproc
temps_Mproc

48.63512349128723

#### La méthode asynchrone est la plus rapide en temps d'éxécution.
#### La méthode multiprocessing est la plus lente 


In [ ]:
#Partie 2

In [22]:
import aiohttp
import asyncio
import websockets
import json
import time
from datetime import datetime

async def get_binance_symbols():
    api_url = 'https://api.binance.com/api/v3/exchangeInfo'
    
    async with aiohttp.ClientSession() as session:
        async with session.get(api_url) as response:
            symbols_data = await response.json()
        
        # Extraction des symboles
        symbols = [symbol['symbol'] for symbol in symbols_data['symbols']]
        
        return symbols

async def process_depth_event(event):
    # Process depth event data as needed
    symbol = event['s']
    time = convert_timestamp_to_date(event['E'])
    bid_price = event['b'][0]
    bid_quantity = event['b'][1]
    ask_price = event['a'][0]
    ask_quantity = event['a'][1]

    # Add the data to your DataFrame or perform any other processing
    data['Symbol'].append(symbol)
    data['Time'].append(time)
    data['Bid Price'].append(bid_price)
    data['Bid Quantity'].append(bid_quantity)
    data['Ask Price'].append(ask_price)
    data['Ask Quantity'].append(ask_quantity)

async def connect_binance_websockets():
    symbols = ['btcusdt', 'btceth']

    async with websockets.connect('wss://stream.binance.com:9443/stream') as ws:
        for symbol in symbols:
            stream_name = f'{symbol}@depth1@100ms'
            subscribe_msg = json.dumps({'method': 'SUBSCRIBE', 'params': [f'{stream_name}'], 'id': 1})
            await ws.send(subscribe_msg)

        while True:
            response = await ws.recv()
            event = json.loads(response)
            
            if 'e' in event and event['e'] == 'depthUpdate':
                await process_depth_event(event)
async def process_depth_event(event, symbol_to_compare='ethusdt'):
    # Process depth event data as needed
    symbol = event['s']
    time = convert_timestamp_to_date(event['E'])
    bid_price = event['b'][0]
    ask_price = event['a'][0]

    # Calculate synthetic bid/ask spread
    synthetic_spread = ask_price - bid_price

    # Display synthetic spread and compare with WebSocket data for ethusdt
    if symbol == symbol_to_compare:
        print(f"{symbol_to_compare} - Time: {time}, Synthetic Spread: {synthetic_spread}")
        print(f"WebSocket Bid Price: {bid_price}, WebSocket Ask Price: {ask_price}")
        print("---")

    # Add the data to your DataFrame or perform any other processing
    data['Symbol'].append(symbol)
    data['Time'].append(time)
    data['Bid Price'].append(bid_price)
    data['Ask Price'].append(ask_price)
    data['Synthetic Spread'].append(synthetic_spread)
#Mise en forme sous un DataFrame
df = pd.DataFrame(data)
df

,Symbol,Time,Close
0,BTCUSDT,2023-11-18 13:51:00,36416.00000000
1,BTCUSDT,2023-11-18 13:52:00,36432.16000000
2,BTCUSDT,2023-11-18 13:53:00,36440.66000000
3,BTCUSDT,2023-11-18 13:54:00,36432.18000000
4,BTCUSDT,2023-11-18 13:55:00,36428.01000000
...,...,...,...
3595,GASBTC,2023-11-18 15:43:00,0.00022890
3596,GASBTC,2023-11-18 15:44:00,0.00022840
3597,GASBTC,2023-11-18 15:45:00,0.00022820
3598,GASBTC,2023-11-18 15:46:00,0.00022840


Arbitrage triangulaire implique l'exploitation d'une disparité dans les taux de change entre trois devises. Dans votre contexte crypto, cela pourrait signifier qu'il y a une opportunité d'arbitrage entre trois paires de trading. Cependant, il y a plusieurs facteurs à prendre en compte, et il est important de noter que les opportunités d'arbitrage peuvent être de courte durée et peuvent disparaître rapidement. Voici une explication générale des facteurs à considérer et des raisons pour lesquelles une opportunité d'arbitrage triangulaire pourrait sembler possible :

Inefficacités du marché : Les opportunités d'arbitrage triangulaire se produisent en raison d'inefficacités temporaires sur le marché. Les échanges de cryptomonnaie sont décentralisés et fonctionnent 24h/24, ce qui peut entraîner des divergences de prix entre différentes plates-formes ou paires de trading.

Frais de transaction : Les frais de transaction sur les plateformes d'échange peuvent affecter la rentabilité de l'arbitrage. Assurez-vous de prendre en compte les frais de transaction, y compris les frais de négociation et de retrait, lors de l'évaluation de l'opportunité d'arbitrage.

Liquidité : L'arbitrage nécessite une liquidité suffisante sur toutes les paires de trading impliquées. Des volumes de négociation faibles peuvent entraîner des difficultés lors de l'exécution des transactions et peuvent également affecter les prix.

Latence du système : Les opportunités d'arbitrage peuvent être de très courte durée, nécessitant une exécution rapide des transactions. La latence du système, y compris les délais de traitement des ordres, peut être un facteur critique.

Variabilité des taux de change : Les taux de change entre différentes paires peuvent fluctuer rapidement. Il est important de surveiller ces variations et de s'assurer que l'arbitrage est toujours rentable.

Facteurs non pris en compte : Les modèles de calcul de l'arbitrage peuvent ne pas tenir compte de tous les facteurs. Par exemple, les frais peuvent varier en fonction du volume de négociation, et certains frais peuvent être cachés.